In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mansarip_dis1_tokens_path = kagglehub.dataset_download('mansarip/dis1-tokens')

print('Data source import complete.')


100%|██████████| 2.67G/2.67G [00:18<00:00, 157MB/s]

Extracting files...


Data source import complete.


In [ ]:
mansarip_dis1_tokens_path

'/root/.cache/kagglehub/datasets/mansarip/dis1-tokens/versions/1'

In [ ]:
# import argparse
# 
# 
# import pickle
# 
# import torch
# from transformers import AutoTokenizer, AutoModel
# from tqdm import tqdm
# 
# from torch.utils.data import Dataset
# 
# from config.config import CONFIG
# 
# class DPRDataset(Dataset):
# 
#     def __init__(self, q_id, lang, tokens):
#         self.q_id = []
#         self.lang = []
#         self.tokens = []
#         for i, chunks in enumerate(tqdm(tokens, desc="Dataset...")):
#             for j in range(len(chunks['input_ids'])):
#                 self.q_id.append(q_id[i])
#                 self.lang.append(lang[i])
#                 chunk_dict = {}
#                 for k, v in chunks.items():
#                     chunk_dict[k] = v[j]
#                 self.tokens.append(chunk_dict)
# 
#     def __getitem__(self, idx):
#         return {
#             "q_id": self.q_id[idx],
#             "lang": self.lang[idx],
#             "tokens": self.tokens[idx]
#         }
# 
#     def __len__(self):
#         return len(self.q_id)
# 
#     def collate_fn(self, batch):
#         return_dict = {}
#         q_ids = [sample["q_id"] for sample in batch]
#         langs = [sample["lang"] for sample in batch]
#         tokens = {k: [sample["tokens"][k] for sample in batch] for k in batch[0]["tokens"].keys()}
#         tokens = {k: torch.stack(v) for k, v in tokens.items()}
# 
#         return_dict["q_id"] = q_ids
#         return_dict["lang"] = langs
#         return_dict["tokens"] = tokens
# 
#         return return_dict
# 
# 
# def pooling(hidden_states, attention_mask):
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
#     sum_embeddings = torch.sum(hidden_states * input_mask_expanded, 1)
#     sum_mask = input_mask_expanded.sum(1)
#     sum_mask = torch.clamp(sum_mask, min=1e-9)
#     mean_embeddings = sum_embeddings / sum_mask
#     return mean_embeddings
# 
# def embed(tokens_path, output_path, split_id) -> None:
# 
# 
#     with open(f"{tokens_path}/tokenized_corpus_{split_id}.pkl", "rb") as f:
#         corpus_i = pickle.load(f)
#     docids_i, langs_i, tokenized_all_chunks_i = corpus_i
#     # Build dataloader
#     dataset = DPRDataset(docids_i, langs_i, tokenized_all_chunks_i)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, collate_fn=dataset.collate_fn, shuffle=False, num_workers=4)
# 
#     model = AutoModel.from_pretrained("microsoft/mdeberta-v3-base").to("cuda" if torch.cuda.is_available() else "cpu")
# 
#     docids_i = []
#     langs_i = []
#     embed_all_chunks_i = []
#     with torch.no_grad():
#         for batch in tqdm(dataloader, desc="Embedding..."):
#             inputs = {k: v.to(model.device) for k, v in batch["tokens"].items()}
#             outputs = model(**inputs, return_dict=True)
#             if CONFIG['use_CLS']:
#                 outputs = outputs['last_hidden_state'][:, 0, :].cpu().numpy()
#             else:
#                 outputs = pooling(outputs['last_hidden_state'], inputs['attention_mask']).cpu().numpy()
#             docids_i += batch["q_id"]
#             langs_i += batch["lang"]
#             embed_all_chunks_i.append(outputs)
# 
#     embed_all_chunks_i = (docids_i, langs_i, embed_all_chunks_i)
#     with open(f"{output_path}/embed_all_chunks_{split_id}.pkl", "wb") as f:
#         pickle.dump(embed_all_chunks_i, f)
# 


In [ ]:
import argparse


import pickle

import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

from torch.utils.data import Dataset

from config.config import CONFIG

print(CONFIG['use_CLS'])

def pooling(hidden_states, attention_mask):
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
    sum_embeddings = torch.sum(hidden_states * input_mask_expanded, 1)
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask
    return mean_embeddings

def embed(tokens_path, output_path, split_id) -> None:

    with open(f"{tokens_path}/tokenized_corpus_{split_id}.pkl", "rb") as f:
        corpus_i = pickle.load(f)
    docids_i, langs_i, tokenized_all_chunks_i = corpus_i
    # Build dataloader

    model = AutoModel.from_pretrained("microsoft/mdeberta-v3-base").to("cuda" if torch.cuda.is_available() else "cpu")

    
    embed_all_chunks_i = []
    embed_all_chunks_i_each = []
    with torch.no_grad():
        for tokens in tqdm(tokenized_all_chunks_i, desc="Embedding..."):
            l = len(tokens['input_ids'])
            if l > CONFIG['max_length']:
                inputs_first = tokens[:l - l % CONFIG['max_length']]
                
                inputs_first = torch.tensor(inputs_first).to(CONFIG['device'])
                inputs_first = inputs_first.reshape((l - l % CONFIG['max_length']) // CONFIG['max_length'], CONFIG['max_length'])
                
                outputs_first = model(input_ids=inputs_first, return_dict=True)
                if CONFIG['use_CLS']:
                    outputs_first = outputs_first['last_hidden_state'][:, 0, :]
                else:
                    # outputs = pooling(outputs['last_hidden_state'], inputs['attention_mask']).cpu().numpy()
                    outputs_first = pooling(outputs_first['last_hidden_state'], torch.ones_like(inputs_first))
                if l % CONFIG['max_length'] == 0:
                    outputs = outputs_first
                else:
                    inputs_last = tokens[l - l % CONFIG['max_length']:]
                
                    inputs_last = torch.tensor(inputs_last).to(CONFIG['device'])
                    inputs_last = inputs_last.reshape(1, -1)
                    
                    outputs_last = model(input_ids=inputs_last, return_dict=True)
                    if CONFIG['use_CLS']:
                        outputs_last = outputs_last['last_hidden_state'][:, 0, :]
                    else:
                        outputs_last = pooling(outputs_last['last_hidden_state'], torch.ones_like(inputs_last))
                    outputs = torch.cat((outputs_first, outputs_last), 0)
                    
                embed_all_chunks_i.append(outputs.mean(0).cpu().numpy())
                embed_all_chunks_i_each.append(outputs.cpu().numpy())
            else:
                inputs = torch.tensor(tokens['input_ids']).to(CONFIG['device'])
                inputs = inputs.reshape(1, -1)
                
                outputs = model(input_ids=inputs, return_dict=True)
                if CONFIG['use_CLS']:
                    outputs = outputs['last_hidden_state'][:, 0, :]
                else:
                    outputs = pooling(outputs['last_hidden_state'], torch.ones_like(inputs))
                embed_all_chunks_i.append(outputs.mean(0).cpu().numpy())
                embed_all_chunks_i_each.append(outputs.cpu().numpy())
                

    embed_all_chunks_i = (docids_i, langs_i, embed_all_chunks_i)
    with open(f"{output_path}/embed_all_chunks_new_{split_id}.pkl", "wb") as f:
        pickle.dump(embed_all_chunks_i, f)
        
    embed_all_chunks_i_each = (docids_i, langs_i, embed_all_chunks_i_each)
    with open(f"{output_path}/embed_all_chunks_each_new_{split_id}.pkl", "wb") as f:
        pickle.dump(embed_all_chunks_i_each, f)


In [ ]:
tokens = mansarip_dis1_tokens_path
# save to drive
output = '/content/drive/MyDrive'
split_ids = [1, 12, 4, 5]
# split_ids = [2, 11, 6, 7]
# split_ids = [3, 10, 8, 9]
for split_id in split_ids:
    embed(tokens, output, split_id)


Dataset...: 100%|██████████| 22336/22336 [00:06<00:00, 3345.92it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Embedding...: 100%|██████████| 1989/1989 [1:19:12<00:00,  2.39s/it]
